# Classifiying if the news pieces are Real or Fake

In [1]:
import pandas as pd
import numpy as np

In [4]:
true= pd.read_csv('/Documents/kaggleDatasets/fakevsrealnews/True.csv')
fake= pd.read_csv('/Documents/kaggleDatasets/fakevsrealnews/Fake.csv')

In [5]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [12]:
true['label']=1
fake['label']=0
dataset=pd.concat([true[['text','title','label']] ,
                   fake[['text','title','label']]])
#shuffle the dataset by using sample method
dataset= dataset.sample(dataset.shape[0]).reset_index()

In [14]:
import pickle
import tensorflow as tf
import tensorflow_hub as hub

import re
import nltk


In [16]:
module_url="https://tfhub.dev/google/universal-sentence-encoder/4"
model= hub.load(module_url)

In [19]:
def embed(input):
    return model([input])
dataset= dataset.head(10000).copy()

In [20]:
dataset['text_vector']=dataset.text.map(embed)
dataset['text_vector']=dataset.text_vector.map(np.array)

dataset['title_vector']=dataset.title.map(embed)
dataset['title_vector']=dataset.title_vector.map(np.array)

pickle.dump(dataset, open('/Documents/kaggleDatasets/dataset_fakenews.pkl','wb'))

In [21]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [30]:
import itertools

xtrain, ytrain, xtest, ytest= dataset[['text_vector']][0:8000],\
        dataset['label'][0:8000], dataset[['text_vector']][8000:],\
    dataset['label'][8000:]
    
xtrain= xtrain.text_vector
xtrain= list(itertools.chain(*xtrain))

xtest= xtest.text_vector
xtest= list(itertools.chain(*xtest))



In [34]:
lr= LogisticRegression(penalty='l2', max_iter=500 , C=1, random_state=42)
#random state ste so that same model can be replicated with same initial condistions
#l2 regularization , C value like SVM , slack parameter, extent of regularization
lr1= lr.fit(xtrain, ytrain)

In [35]:
lr_predict= lr.predict(xtest)

lr_predict_report= classification_report( ytest, lr_predict)

print(lr_predict_report)

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1062
           1       0.93      0.95      0.94       938

    accuracy                           0.94      2000
   macro avg       0.94      0.94      0.94      2000
weighted avg       0.94      0.94      0.94      2000



In [36]:
lr_cm= confusion_matrix(ytest, lr_predict)
print(lr_cm)

[[998  64]
 [ 49 889]]
